In [1]:
%run CC.ipynb

In [2]:
OrderLog20020812_df = pd.read_csv('./resources/OrderLog20020812.txt')
OrderLog20020812_df.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,NaN,NaN
1,2,RU0009071187,S,103000000,2,1,990.000,300,NaN,NaN
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,NaN,NaN
3,4,RU0009100762,B,103000000,4,1,0.133,100000,NaN,NaN
4,5,RU0009054449,S,103000000,5,1,332.000,500,NaN,NaN


In [3]:
dic = {4434:0, 'asdf':1}
dic[4434], dic['asdf'], len(dic)

(0, 1, 2)

In [4]:
class TAQ2TAQpps:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col # orderno or some mark linking trades and quotes
             ):
        self.taq_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.secs = pd.unique(taq_df[sec_col])
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        self.rows_to_edges()
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.TAQpps = {k:v for (k, v) in zip(self.secs, [TAQplots(self.Gs[sec]) for sec in self.secs])}

    def sec_rowid_lnk2edges(self, rowid, sec, lnk): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk) 
         for rowid, sec, lnk in zip(self.taq_df[self.row_id_col],
                                    self.taq_df[self.sec_col],
                                    self.taq_df[self.link_col])]
        

In [5]:
temp = TAQ2TAQpps(OrderLog20020812_df, 'NO', 'SECCODE', 'ORDERNO')
[print(temp.TAQpps[sec].to_df()) for sec in temp.secs]

NameError: name 'taq2CCs' is not defined